In [1]:
from datasets import Dataset, DatasetDict
import pandas as pd

def read_conll(filepath):
    sentences = []
    labels = []
    with open(filepath, encoding='utf-8') as f:
        tokens = []
        ner_tags = []
        for line in f:
            line = line.strip()
            if not line:
                if tokens:
                    sentences.append(tokens)
                    labels.append(ner_tags)
                    tokens = []
                    ner_tags = []
            else:
                splits = line.split()
                if len(splits) < 2:
                    print(f"Skipping malformed line: {line}")
                    continue
                tokens.append(splits[0])
                ner_tags.append(splits[1])
        if tokens:
            sentences.append(tokens)
            labels.append(ner_tags)
    return sentences, labels


tokens, ner_tags = read_conll("../labeled/telegram_conll_sample.txt")


e:\Kifya\ethioMart-amharic-ner\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Skipping malformed line: ፈቺI-ProductO


In [2]:
# Get unique labels
unique_labels = list(set(tag for doc in ner_tags for tag in doc))
label2id = {label: idx for idx, label in enumerate(sorted(unique_labels))}
id2label = {idx: label for label, idx in label2id.items()}

# Encode labels
encoded_tags = [[label2id[tag] for tag in doc] for doc in ner_tags]

# Create dataset
dataset = Dataset.from_dict({"tokens": tokens, "ner_tags": encoded_tags})

# Optional: Split into train/test
dataset = dataset.train_test_split(test_size=0.2)


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("xlm-roberta-base")

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        padding='max_length',
        max_length=128,
    )
    
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                # Use label or -100 for sub-tokens
                label_ids.append(label[word_idx])
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)


Map: 100%|██████████| 5/5 [00:00<00:00, 31.32 examples/s]


In [4]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    "xlm-roberta-base",
    num_labels=len(label2id),
    id2label=id2label,
    label2id=label2id,
)

training_args = TrainingArguments(
    output_dir="./amharic-ner-checkpoints",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


KeyboardInterrupt: 

In [ ]:
trainer.train()
trainer.save_model("amharic_ner_finetuned_model")
